In [1]:
# https://www.energate-messenger.de/markt/strom/preisgruppe/174994/eex-phelix-de-futures-base

import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
# get the content of the website
page = requests.get("https://www.energate-messenger.de/markt/strom/preisgruppe/174994/eex-phelix-de-futures-base")
html = page.content

In [3]:
# parse the html and save it into a BeautifulSoup instance
bs = BeautifulSoup(html, 'html.parser')

In [4]:
bs.find('thead').find_all('th')

[<th colspan="2">Preisreihe</th>,
 <th>Datum</th>,
 <th>Preis</th>,
 <th>Preis Tief</th>,
 <th>Preis Hoch</th>,
 <th>Umsatz</th>,
 <th>Open Interest</th>,
 <th>No. of Trades</th>]

In [5]:
pricetable = bs.find('tbody')
pricetable.find_all('td')

[<td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175102/eex-phelix-de-baseload-cal-2023">Jahr 2023</a></td>,
 <td class="date">27.04.2022</td>,
 <td class="numeric">202,06</td>,
 <td class="numeric">197,00</td>,
 <td class="numeric">212,50</td>,
 <td class="numeric">1.235.160</td>,
 <td class="numeric">69.065</td>,
 <td class="numeric">114</td>,
 <td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175103/eex-phelix-de-baseload-cal-2024">Jahr 2024</a></td>,
 <td class="date">27.04.2022</td>,
 <td class="numeric">167,50</td>,
 <td class="numeric">166,00</td>,
 <td class="numeric">170,25</td>,
 <td class="numeric">325.008</td>,
 <td class="numeric">16.756</td>,
 <td class="numeric">33</td>,
 <td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175104/eex-phelix-de-baseload-cal-2025">Jahr 2025</a></td>,
 <td class="date">27.04.2022</td>,
 <td class="numeric">145,13</td>,
 <td class="numeric">144,00</td>,
 <td class="numeric">145,00</td>,
 <td class="numeric

In [6]:
pricetable.find_all(class_="numeric")

[<td class="numeric">202,06</td>,
 <td class="numeric">197,00</td>,
 <td class="numeric">212,50</td>,
 <td class="numeric">1.235.160</td>,
 <td class="numeric">69.065</td>,
 <td class="numeric">114</td>,
 <td class="numeric">167,50</td>,
 <td class="numeric">166,00</td>,
 <td class="numeric">170,25</td>,
 <td class="numeric">325.008</td>,
 <td class="numeric">16.756</td>,
 <td class="numeric">33</td>,
 <td class="numeric">145,13</td>,
 <td class="numeric">144,00</td>,
 <td class="numeric">145,00</td>,
 <td class="numeric">43.800</td>,
 <td class="numeric">4.657</td>,
 <td class="numeric">5</td>,
 <td class="numeric">241,25</td>,
 <td class="numeric">238,00</td>,
 <td class="numeric">259,00</td>,
 <td class="numeric">704.352</td>,
 <td class="numeric">191.593</td>,
 <td class="numeric">232</td>,
 <td class="numeric">261,00</td>,
 <td class="numeric">256,00</td>,
 <td class="numeric">272,70</td>,
 <td class="numeric">386.575</td>,
 <td class="numeric">181.525</td>,
 <td class="numeric">1

---

## Lists

### PriceSeries
First we store the original header in a list. After we adapt our own formatting.

In [45]:
headers = pricetable.find_all('a')
headers_lst = [header.get_text() for header in headers]
headers_lst[::5]

['Jahr 2023', 'Quartal 1/2023', 'Quartal 2/2024', 'Juni 2022', 'November 2022']

In [54]:
headers_lst = [sub.replace('20', '') for sub in headers_lst]
headers_lst = [sub.replace('Jahr ', 'cal-') for sub in headers_lst]
headers_lst = [sub.replace('Quartal ', 'Q') for sub in headers_lst]
headers_lst = [sub.replace('/', '-') for sub in headers_lst]
headers_lst = [sub.replace('Januar ', 'M1-') for sub in headers_lst]
headers_lst = [sub.replace('Februar ', 'M2-') for sub in headers_lst]
headers_lst = [sub.replace('März ', 'M3-') for sub in headers_lst]
headers_lst = [sub.replace('Maerz ', 'M3-') for sub in headers_lst]
headers_lst = [sub.replace('April ', 'M4-') for sub in headers_lst]
headers_lst = [sub.replace('Mai ', 'M5-') for sub in headers_lst]
headers_lst = [sub.replace('Juni ', 'M6-') for sub in headers_lst]
headers_lst = [sub.replace('Juli ', 'M7-') for sub in headers_lst]
headers_lst = [sub.replace('August ', 'M8-') for sub in headers_lst]
headers_lst = [sub.replace('September ', 'M9-') for sub in headers_lst]
headers_lst = [sub.replace('Oktober ', 'M10-') for sub in headers_lst]
headers_lst = [sub.replace('November ', 'M11-') for sub in headers_lst]
headers_lst = [sub.replace('Dezember ', 'M12-') for sub in headers_lst]
headers_lst[::5]

['cal-23', 'Q1-23', 'Q2-24', 'M6-22', 'M11-22']

In [53]:
from datetime import datetime as dt 
mn = 'Februar'
dt_obj = dt.strptime(mn, '%B')
mn = dt_obj.month
print(mn)

ValueError: time data 'Februar' does not match format '%B'

In [35]:
types = pricetable.find_all(class_='type')
types_lst = [typ.get_text() for typ in types]
types_lst[::5]

['Jahr', 'Quartal', 'Quartal', 'Monat', 'Monat']

In [193]:
dates = pricetable.find_all(class_="date")
dates_lst = [date.get_text() for date in dates]
dates_lst

['27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022',
 '27.04.2022']

In [194]:
numerics = pricetable.find_all(class_='numeric')
numerics_lst = [numeric.get_text() for numeric in numerics]
numerics_lst

['202,06',
 '197,00',
 '212,50',
 '1.235.160',
 '69.065',
 '114',
 '167,50',
 '166,00',
 '170,25',
 '325.008',
 '16.756',
 '33',
 '145,13',
 '144,00',
 '145,00',
 '43.800',
 '4.657',
 '5',
 '241,25',
 '238,00',
 '259,00',
 '704.352',
 '191.593',
 '232',
 '261,00',
 '256,00',
 '272,70',
 '386.575',
 '181.525',
 '125',
 '249,63',
 '245,00',
 '253,00',
 '103.632',
 '12.339',
 '32',
 '176,00',
 '172,50',
 '176,50',
 '24.024',
 '5.216',
 '9',
 '175,00',
 '175,00',
 '175,00',
 '11.040',
 '4.850',
 '2',
 '208,37',
 '208,00',
 '209,00',
 '30.926',
 '4.419',
 '5',
 '207,39',
 '200,00',
 '200,00',
 '13.098',
 '471',
 '2',
 '145,90',
 '-',
 '-',
 '-',
 '327',
 '-',
 '145,00',
 '-',
 '-',
 '-',
 '425',
 '-',
 '171,93',
 '-',
 '-',
 '-',
 '78',
 '-',
 '166,32',
 '-',
 '-',
 '-',
 '236.941',
 '-',
 '215,25',
 '211,49',
 '250,00',
 '508.152',
 '227.750',
 '350',
 '224,00',
 '218,50',
 '249,00',
 '212.400',
 '215.904',
 '157',
 '231,31',
 '235,00',
 '252,50',
 '12.648',
 '890',
 '7',
 '228,21',
 '239,

In [195]:
len(numerics_lst)
slst = numerics_lst
slst = ([s.replace('.','') for s in slst])
slst = ([s.replace(',','.') for s in slst])

slst


['202.06',
 '197.00',
 '212.50',
 '1235160',
 '69065',
 '114',
 '167.50',
 '166.00',
 '170.25',
 '325008',
 '16756',
 '33',
 '145.13',
 '144.00',
 '145.00',
 '43800',
 '4657',
 '5',
 '241.25',
 '238.00',
 '259.00',
 '704352',
 '191593',
 '232',
 '261.00',
 '256.00',
 '272.70',
 '386575',
 '181525',
 '125',
 '249.63',
 '245.00',
 '253.00',
 '103632',
 '12339',
 '32',
 '176.00',
 '172.50',
 '176.50',
 '24024',
 '5216',
 '9',
 '175.00',
 '175.00',
 '175.00',
 '11040',
 '4850',
 '2',
 '208.37',
 '208.00',
 '209.00',
 '30926',
 '4419',
 '5',
 '207.39',
 '200.00',
 '200.00',
 '13098',
 '471',
 '2',
 '145.90',
 '-',
 '-',
 '-',
 '327',
 '-',
 '145.00',
 '-',
 '-',
 '-',
 '425',
 '-',
 '171.93',
 '-',
 '-',
 '-',
 '78',
 '-',
 '166.32',
 '-',
 '-',
 '-',
 '236941',
 '-',
 '215.25',
 '211.49',
 '250.00',
 '508152',
 '227750',
 '350',
 '224.00',
 '218.50',
 '249.00',
 '212400',
 '215904',
 '157',
 '231.31',
 '235.00',
 '252.50',
 '12648',
 '890',
 '7',
 '228.21',
 '239.00',
 '239.00',
 '744',
 '

In [196]:
def make_sublists(main_list, sublist_size):
    return [main_list[x:x+sublist_size]
            for x in range(0, len(main_list), sublist_size)]

In [197]:
def extract(lst, ii):
    return [item[ii] for item in lst]

In [198]:
sublist = extract(make_sublists(slst, 6),0)
sublist

['202.06',
 '167.50',
 '145.13',
 '241.25',
 '261.00',
 '249.63',
 '176.00',
 '175.00',
 '208.37',
 '207.39',
 '145.90',
 '145.00',
 '171.93',
 '166.32',
 '215.25',
 '224.00',
 '231.31',
 '228.21',
 '265.00',
 '257.17',
 '269.73',
 '256.39',
 '268.22']

In [199]:
len(slst)/6

23.0

In [200]:
df = pd.DataFrame(list(zip(headers_lst, types_lst, dates_lst)), columns=['PriceSeries', 'PriceType', 'Date'])

# Numeric Columns
hlist = ['Price', 'PriceLow', 'PriceHigh', 'Volumne', 'OpenInterest', 'TradesCount']
for i in range(len(hlist)):
    df[hlist[i]] = extract(make_sublists(slst, 6),i)
    
df

,PriceSeries,PriceType,Date,Price,PriceLow,PriceHigh,Volumne,OpenInterest,TradesCount
0,Jahr 2023,Jahr,27.04.2022,202.06,197.00,212.50,1235160,69065,114
1,Jahr 2024,Jahr,27.04.2022,167.50,166.00,170.25,325008,16756,33
2,Jahr 2025,Jahr,27.04.2022,145.13,144.00,145.00,43800,4657,5
3,Quartal 3/2022,Quartal,27.04.2022,241.25,238.00,259.00,704352,191593,232
4,Quartal 4/2022,Quartal,27.04.2022,261.00,256.00,272.70,386575,181525,125
5,Quartal 1/2023,Quartal,27.04.2022,249.63,245.00,253.00,103632,12339,32
6,Quartal 2/2023,Quartal,27.04.2022,176.00,172.50,176.50,24024,5216,9
7,Quartal 3/2023,Quartal,27.04.2022,175.00,175.00,175.00,11040,4850,2
8,Quartal 4/2023,Quartal,27.04.2022,208.37,208.00,209.00,30926,4419,5
9,Quartal 1/2024,Quartal,27.04.2022,207.39,200.00,200.00,13098,471,2


In [201]:
'eex_'+df.Date[0].replace('.','-')

'eex_27-04-2022'

In [202]:
df.to_csv('eex_'+df.Date[0].replace('.','-') + '.csv', index=False)